# Importando libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, Dropdown
import plotly.express as px
import os

# Lendo o dataset

In [2]:
try:   
   df_processed = pd.read_csv('../data/prouni_2005_2019_processed.csv')
except Exception as e:
    print(f"Error: {e}")

#df_processed.head()

In [3]:
lista_colunas = df_processed.columns
lista_colunas

Index(['ANO_CONCESSAO', 'INSTITUICAO', 'TIPO', 'MODALIDADE_ENSINO', 'CURSO',
       'TURNO', 'GENERO', 'RACA', 'DEFICIENTE_FISICO', 'REGIAO', 'UF',
       'MUNICIPIO', 'IDADE'],
      dtype='object')

# Gráficos

Criando pasta charts

In [4]:
df_chars = df_processed.copy()

In [5]:

#lista_renomeada = [col.replace("_", " ").upper() for col in lista_colunas]
lista_renomeada = ['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA','MODALIDADE DE ENSINO','CURSO','TURNO','GÊNERO','RAÇA','DEFICIENTE FÍSICO','REGIÃO','UF','MUNICÍPIO','IDADE']
df_chars.rename(columns=dict(zip(lista_colunas, lista_renomeada)), inplace=True)
df_chars.columns

Index(['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA',
       'MODALIDADE DE ENSINO', 'CURSO', 'TURNO', 'GÊNERO', 'RAÇA',
       'DEFICIENTE FÍSICO', 'REGIÃO', 'UF', 'MUNICÍPIO', 'IDADE'],
      dtype='object')

In [6]:
os.makedirs("../figures", exist_ok=True)

## Número de bolsas por ano

In [7]:
anos = sorted(df_chars["ANO DE CONCESSÃO"].unique())
dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)
output_tipo_bolsas_ano = widgets.Output()
def plot_tipo_bolsas_ano(ano):
    import os
    with output_tipo_bolsas_ano:
        output_tipo_bolsas_ano.clear_output()

    dados = df_chars[df_chars["ANO DE CONCESSÃO"] == ano]
    # contagem de tipos de bolsa nesse ano
    contagem = (
        dados["TIPO DE BOLSA"]
        .value_counts()
        .reset_index()
    )
    contagem.columns = ["TIPO DE BOLSA", "QUANTIDADE"]
    
    fig = px.bar(
        contagem,
        x="TIPO DE BOLSA",
        y="QUANTIDADE",
        text="QUANTIDADE",
        title=f"Quantidade de cada tipo de bolsa concedida em {ano}",
        color='TIPO DE BOLSA',
    )
    
    # deixa o texto em cima da barra e melhora layout
    fig.update_traces(textposition="outside")
    fig.update_layout(
        height= 650,
        xaxis_title="TIPO DE BOLSA",
        yaxis_title="QUANTIDADE",
        uniformtext_minsize=2,
        uniformtext_mode="hide",
        
    )
    
    fig.show()
    os.makedirs("../figures/tipo_bolsas_ano", exist_ok=True)
    fig.write_image(f"../figures/tipo_bolsas_ano/fig_tipo_bolsas_{ano}.png")


widgets.interact(plot_tipo_bolsas_ano, ano=dropdown_ano);

interactive(children=(Dropdown(description='ANO', options=(np.int64(2005), np.int64(2006), np.int64(2007), np.…

## Número de bolsas por sexo

In [ ]:
anos = ['TODOS OS ANOS']
anos.extend(sorted(df_chars["ANO DE CONCESSÃO"].unique()))

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)

df_idade_genero = df_chars.loc[df_chars["IDADE"].between(10, 70)].copy()

bins = list(range(10, 75, 5))
labels = [f"{i}-{i+4}" for i in range(10, 70, 5)]

df_idade_genero["FAIXA DE IDADE"] = pd.cut(
    df_idade_genero["IDADE"],
    bins=bins,
    labels=labels,
    right=False
)

faixas = sorted(df_idade_genero["FAIXA DE IDADE"].dropna().unique().astype(str))

dropdown_faixa = widgets.Dropdown(
    options=["TODAS AS IDADES"] + faixas,
    value="TODAS AS IDADES",
    description="FAIXA DE IDADE"
)
dropdown_ano.style = {"description_width": "initial"}
dropdown_faixa.style = {"description_width": "initial"}

def plot_total_idade_e_genero(ano, faixa_idade):
    base = df_idade_genero[df_idade_genero["ANO DE CONCESSÃO"] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_idade_genero

    if faixa_idade != "TODAS AS IDADES":
        base = base[base["FAIXA DE IDADE"].astype(str) == faixa_idade]

    if base.empty:
        print("Sem dados para esse filtro.")
        return

    agrupado = (
        base.groupby(["TIPO DE BOLSA", "GÊNERO"])
            .size()
            .reset_index(name="QUANTIDADE")
    )

    tabela = agrupado.pivot_table(
        index="TIPO DE BOLSA",
        columns="GÊNERO",
        values="QUANTIDADE",
        fill_value=0,
    ).reset_index()

    if "F" not in tabela.columns:
        tabela["F"] = 0
    if "M" not in tabela.columns:
        tabela["M"] = 0

    tabela["TOTAL"] = tabela["F"] + tabela["M"]

    tabela = tabela.rename(columns={
        "F": "FEMININO",
        "M": "MASCULINO"
    })

    fig = px.bar(
        tabela,
        x="TIPO DE BOLSA",
        y=["TOTAL", "FEMININO", "MASCULINO"],
        barmode="group",
        title=f"Quantidade de bolsas por tipo e gênero em {ano.lower()}" if ano == 'TODOS OS ANOS' else f"Quantidade de bolsas por tipo e gênero em {ano}",
        text_auto=True,
        color_discrete_map={
            "TOTAL": "#5C6159",
            "FEMININO": "#e83e8c",
            "MASCULINO": "#1f77b4"
        }
    )

    fig.update_traces(
    hovertemplate=
        "TIPO: %{x}<br>"
        + "QTDE: %{y:,.0f}".replace(",", ".")
        + "<extra></extra>",
        textposition="outside",
        texttemplate="%{y:,.0f}".replace(",", ".")
    )


    
    fig.update_layout(
        height=600,
        width=1000,
        xaxis_title="TIPO DE BOLSA",
        yaxis_title="QUANTIDADE DE BOLSAS",
        uniformtext_minsize=6,
        uniformtext_mode="hide",
        legend_title="LEGENDA"
    )

    fig.show()

    os.makedirs("../figures/bolsas_por_genero", exist_ok=True)
    fig.write_image(
        f"../figures/bolsas_por_genero/fig_tipo_bolsas_{ano}_{faixa_idade}.png"
    )

dropdown_ano.layout = widgets.Layout(width="175px")
dropdown_faixa.layout = widgets.Layout(width="250px")

ui = widgets.HBox(
    [dropdown_ano, dropdown_faixa],
     layout=widgets.Layout(gap="15px")
    )

out = widgets.interactive_output(
    plot_total_idade_e_genero,
    {
        "ano": dropdown_ano,
        "faixa_idade": dropdown_faixa
    }
)

display(ui, out)


Output()

## Crescimento de cada tipo de bolsa ao longo dos anos

In [9]:
# Paleta opcional por tipo
cores_por_tipo = {
    "BOLSA INTEGRAL": "#1f77b4",
    "BOLSA PARCIAL 50%": "#ff7f0e",
    "BOLSA COMPLEMENTAR 25%": "#2ca02c"
}

# Dropdowns
tipos = sorted(df_chars["TIPO DE BOLSA"].unique())
dropdown_tipo = widgets.Dropdown(options=tipos, value=tipos[0], description="TIPO DE BOLSA")

# PAÍS = agregado, Comparar regiões = várias linhas
regioes = ["PAÍS", "COMPARAR REGIÕES"] + sorted(df_chars["REGIÃO"].unique())

dropdown_regiao = widgets.Dropdown(options=regioes, value="PAÍS", description="REGIÃO")

modos = ["QUANTIDADE", "CRESCIMENTO PERCENTUAL"]
dropdown_modo = widgets.Dropdown(options=modos, value="QUANTIDADE", description="ESCALA")


def plot_crescimento(tipo, regiao, modo):
    base = df_chars[df_chars["TIPO DE BOLSA"] == tipo].copy()
    if base.empty:
        print("Sem dados para esse tipo.")
        return

    # Cenário 1: PAÍS (agregado)
    if regiao == "PAÍS":
        dados = (
            base.groupby("ANO DE CONCESSÃO")
                .size()
                .reset_index(name="QUANTIDADE")
                .sort_values("ANO DE CONCESSÃO")
        )
        dados["TIPO DE BOLSA"] = tipo

        primeiro_valor = dados["QUANTIDADE"].iloc[0]
        if primeiro_valor == 0:
            dados["Crescimento_percentual"] = pd.NA
        else:
            dados["Crescimento_percentual"] = (dados["QUANTIDADE"] / primeiro_valor - 1) * 100

        if modo == "QUANTIDADE":
            metrica = "QUANTIDADE"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano (%)"

        cor_tipo = cores_por_tipo.get(tipo, "#1f77b4")

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            markers=True,
            title=f"Evolução de {tipo} no país",
            color_discrete_sequence=[cor_tipo],
            hover_data={
                "ANO DE CONCESSÃO": True,
                "TIPO DE BOLSA": True,
                "QUANTIDADE": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "QUANTIDADE" else False,
            }
        )

    # Cenário 2: região específica
    elif regiao in base["REGIÃO"].unique():
        base = base[base["REGIÃO"] == regiao]
        if base.empty:
            print("Sem dados para essa região.")
            return

        dados = (
            base.groupby("ANO DE CONCESSÃO")
                .size()
                .reset_index(name="QUANTIDADE")
                .sort_values("ANO DE CONCESSÃO")
        )
        dados["TIPO DE BOLSA"] = tipo
        dados["REGIÃO"] = regiao

        primeiro_valor = dados["QUANTIDADE"].iloc[0]
        if primeiro_valor == 0:
            dados["Crescimento_percentual"] = pd.NA
        else:
            dados["Crescimento_percentual"] = (dados["QUANTIDADE"] / primeiro_valor - 1) * 100

        if modo == "QUANTIDADE":
            metrica = "QUANTIDADE"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano (%)"

        cor_tipo = cores_por_tipo.get(tipo, "#1f77b4")

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            markers=True,
            title=f"Evolução de {tipo} na região {regiao}",
            color_discrete_sequence=[cor_tipo],
            hover_data={
                "ANO DE CONCESSÃO": True,
                "TIPO DE BOLSA": True,
                "REGIÃO": True,
                "QUANTIDADE": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "QUANTIDADE" else False,
            }
        )

    # Cenário 3: Comparar regiões
    elif regiao == "COMPARAR REGIÕES":
        dados = (
            base.groupby(["ANO DE CONCESSÃO", "REGIÃO"])
                .size()
                .reset_index(name="QUANTIDADE")
                .sort_values(["REGIÃO", "ANO DE CONCESSÃO"])
        )

        # Crescimento percentual por região
        def calc_crescimento(s):
            if s.iloc[0] == 0:
                return pd.Series([pd.NA] * len(s), index=s.index)
            return (s / s.iloc[0] - 1) * 100

        dados["Crescimento_percentual"] = (
            dados.groupby("REGIÃO")["QUANTIDADE"].transform(calc_crescimento)
        )

        if modo == "QUANTIDADE":
            metrica = "QUANTIDADE"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano de cada região (%)"

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            color="REGIÃO",
            markers=True,
            title=f"Evolução de {tipo} comparando regiões",
            hover_data={
                "ANO DE CONCESSÃO": True,
                "REGIÃO": True,
                "QUANTIDADE": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "QUANTIDADE" else False,
            }
        )

    else:
        print("Opção de região inválida.")
        return

    fig.update_layout(
        xaxis_title="ANO DE CONCESSÃO",
        yaxis_title=y_label,
        xaxis=dict(dtick=1),
        height=650
    )
    fig.show()
    os.makedirs("../figures/crescimento", exist_ok=True)
    fig.write_image(f"../figures/crescimento/{tipo.lower().replace(' ', '_')}_{regiao.lower().replace(' ', '_')}_{modo.lower().replace(' ', '_')}.png")

ui = widgets.HBox([dropdown_tipo, dropdown_regiao, dropdown_modo])
out = widgets.interactive_output(
    plot_crescimento,
    {
        "tipo": dropdown_tipo,
        "regiao": dropdown_regiao,
        "modo": dropdown_modo
    }
)
display(ui, out)

Output()

## Mapa

In [10]:
import json
# Carrega o geojson local (sem internet)
with open("brazil-states.geojson", encoding="utf-8") as f:
   geojson = json.load(f)

# Dropdown de anos
anos = sorted(df_chars["ANO DE CONCESSÃO"].unique())
dropdown_ano_mapa = widgets.Dropdown(
   options=anos,
   value=anos[0],
   description="ANO"
)

# Dropdown de tipos (com opção "Todos")
tipos = ["Todos os tipos"] + sorted(df_chars["TIPO DE BOLSA"].unique())
dropdown_tipo_mapa = widgets.Dropdown(
   options=tipos,
   value="Todos os tipos",
   description="TIPO"
)

def mapa_bolsas_por_uf(ano, tipo):
   # Filtra por ano
   dados = df_chars[df_chars["ANO DE CONCESSÃO"] == ano].copy()
   
   # Filtra por tipo, se não for "Todos"
   if tipo != "Todos os tipos":
      dados = dados[dados["TIPO DE BOLSA"] == tipo]
   
   if dados.empty:
      print("Sem dados para essa combinação de ano e tipo.")
      return
   
   # Agrupa por UF e REGIAO
   mapa_df = (
      dados.groupby(["UF", "REGIÃO"])
            .size()
            .reset_index(name="TOTAL")
            .sort_values("TOTAL", ascending=False)
   )

   # Título dinâmico
   if tipo == "Todos os tipos":
      titulo = f"Total de bolsas concedidas por estado em {ano}"
   else:
      titulo = f"Total de bolsas de {tipo} por estado em {ano}"

   # Mapa coroplético por estado
   fig = px.choropleth(
      mapa_df,
      geojson=geojson,
      locations="UF",                  # sua coluna de UF, tipo "SC", "SP"
      featureidkey="properties.sigla", # campo de UF dentro do geojson
      color="TOTAL",
      color_continuous_scale="Viridis",
      hover_data={"UF": True, "REGIÃO": True, "TOTAL": True},
      title=titulo,
   )

   fig.update_geos(
      fitbounds="locations",
      visible=False
   )

   fig.update_layout(
      height=500,
      coloraxis_colorbar=dict(title="TOTAL DE BOLSAS"),
      coloraxis_showscale=True,
      margin={"r":0, "t":50, "l":0, "b":0},
      width=1000,
   ),

   fig.show()
   os.makedirs("../figures/bolsas_por_uf", exist_ok=True)
   fig.write_image(f"../figures/bolsas_por_uf/{ano}_{tipo.replace(' ', '_').lower()}.png")
   

widgets.interact(
   mapa_bolsas_por_uf,
   ano=dropdown_ano_mapa,
   tipo=dropdown_tipo_mapa
);

interactive(children=(Dropdown(description='ANO', options=(np.int64(2005), np.int64(2006), np.int64(2007), np.…